# clean data 

## importing library

In [1]:
import pandas as pd


In [3]:
# load the csv file 

df = pd.read_csv("cybersecurity_attacks.csv")

## display information 

In [6]:
df.head(5)

In [7]:
df.describe()

In [8]:
df.info()

## information about the table 

- It has 40,000 rows and 25 columns.
- TimeStamps is a data type column 
- Several columns have missing values (e.g., Malware Indicators, Alerts/Warnings, Proxy Information,Firewall logs, IDS/IPS Alerts).
- numerical column ( Packet Length, Anamaly Scores) numerical analyses can be made on it 
- string column (Source IP Address, Destination IP Address, Source Port, Destination Port, Payload Data, User information, Device Information, Geo-location Data )
- Some categorical features (e.g., Protocol, Packet Type, Traffic Type,Attack Type,Attack Signature,Action Taken,  Severity Level, Network Segment, Log Source) will require encoding for ML.
- columns we will transform to categorial (Malware Indicators, Alerts/warnings, Firewall logs, IDS/IPS Alerts) 
- Proxy Information is a string column , but will be removed not relevant for our prediction and has many NaN value more than half of our set  

## missing column values

### Malware Indicator


In [11]:
## ading a value "not ioc detected" to the Malware Indicators column where the value is NaN

df['Malware Indicators'] = df['Malware Indicators'].fillna('No IoC Detected')

### Alerts/Warnings

In [12]:
# adding a value "No Alert triggered" to the Alerts/Warnings column where the value is NaN

df['Alerts/Warnings'] = df['Alerts/Warnings'].fillna('No Alert Triggered')

### Proxy Information 
     This column is not really relevant for our model and has many nan 

In [13]:
# Delete Proxy Information column

df = df.drop('Proxy Information', axis=1)

### FireWall Logs


In [14]:
# adding a value "No Log Data" to the Firewall logs column where the value is NaN

df['Firewall Logs'] = df['Firewall Logs'].fillna('No Log Data')

### IDS/IPS Alerts 

In [15]:
# adding a value "No Alert Data" to the IDS/IPS Alerts column where the value is NaN

df['IDS/IPS Alerts'] = df['IDS/IPS Alerts'].fillna('No Alert Data')

In [16]:
## we verify that the changes have been made

df.info()

## Data conversion 

### timestamps column 


In [17]:
## timestamp column is not in the right format, we will convert it to datetime format, done separately for pandas 

df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [18]:
df.info()

In [24]:
# Dictionary defining column names and their data types
column_dtypes = {   
    "Source IP Address":'string',
    "Destination IP Address": 'string',
    "Source Port": 'string',
    "Destination Port": 'string',
    "Protocol": "category",
    "Packet Length": int,
    "Packet Type": "category",
    "Traffic Type": "category",
    "Payload Data": 'string',
    "Malware Indicators": "category",
    "Anomaly Scores": float,
    "Alerts/Warnings": "category",
    "Attack Type": "category",
    "Attack Signature": "category",
    "Action Taken": "category",
    "Severity Level": "category",
    "User Information": 'string',
    "Device Information": 'string',
    "Network Segment": "category",
    "Geo-location Data": 'string',  
    "Firewall Logs": "category",
    "IDS/IPS Alerts": "category",
    "Log Source": "category"
}

# Convert all other columns based on dictionary
for col, dtype in column_dtypes.items():
    if dtype == str:
        df[col] = df[col].astype("string")  # Explicitly enforce str type
    else:
        df[col] = df[col].astype(dtype)

In [25]:
df.info()

## storing the cleaned data frame in csv 

In [27]:
## we will now store the cleaned data in a new csv file

df.to_csv("cleaned_cybersecurity_attacks.csv", index=False)

# Outliers 

## graphs to detect some outliers 

# Feature Eng

## import the cleaned csv 

In [46]:
df_cleaned = pd.read_csv("cleaned_cybersecurity_attacks.csv")
df_cleaned.head(5)

In [21]:
df_cleaned_copy = df_cleaned.copy()
df_cleaned_copy.head(5)

add a column attack_id

In [47]:
df_cleaned_copy['attack_Index'] = df_cleaned_copy.index
df_cleaned_copy.head(5)

for our FE, to feed in numerical data to our model we will need to drop some non numerical columns like :
- Timestamp
- Source IP Address
- Destination Address
- Source Port 
- Destination Port
- Pay Load 
- User Information
- Device Information
- Geo-location Data


In [23]:
def drop_columns(df):
    columns_to_drop = ["Timestamp", "Source IP Address", "Destination IP Address", "Source Port", "Destination Port", "Payload Data", "User Information", "Device Information", "Geo-location Data"]
    df = df.drop(columns=columns_to_drop)
    return df

df_cleaned_copy = drop_columns(df_cleaned_copy)
df_cleaned_copy.head(5)

In [ ]:
df_cleaned_copy = df_cleaned_copy.drop(columns=["Index"])

In [14]:
df_cleaned_copy.to_csv("cybersecurity_attacks_index.csv", index=False)

In [21]:
new_df = pd.read_csv("cybersecurity_attacks_index.csv", index_col="attack_Index")
new_df.head(5)

,Protocol,Packet Length,Packet Type,Traffic Type,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Type,Attack Signature,Action Taken,Severity Level,Network Segment,Firewall Logs,IDS/IPS Alerts,Log Source
attack_Index,,,,,,,,,,,,,,,
0,ICMP,503,Data,HTTP,IoC Detected,28.67,No Alert Triggered,Malware,Known Pattern B,Logged,Low,Segment A,Log Data,No Alert Data,Server
1,ICMP,1174,Data,HTTP,IoC Detected,51.50,No Alert Triggered,Malware,Known Pattern A,Blocked,Low,Segment B,Log Data,No Alert Data,Firewall
2,UDP,306,Control,HTTP,IoC Detected,87.42,Alert Triggered,DDoS,Known Pattern B,Ignored,Low,Segment C,Log Data,Alert Data,Firewall
3,UDP,385,Data,HTTP,No IoC Detected,15.79,Alert Triggered,Malware,Known Pattern B,Blocked,Medium,Segment B,No Log Data,Alert Data,Firewall
4,TCP,1462,Data,DNS,No IoC Detected,0.52,Alert Triggered,DDoS,Known Pattern B,Blocked,Low,Segment C,No Log Data,Alert Data,Firewall



implicite Int data type for Packet Length  , no extreme outliers for this column
implicite Int casting for Anomaly Score


In [ ]:
def cast_int_columns(df): ## didn't use this one beccause it takes the attack_Index as a column 
    df['Packet Length'] = df['Packet Length'].astype(int)
    df['Anomaly Scores'] = df['Anomaly Scores'].astype(float)
    return df

# Apply the function to the dataframe
new_df = cast_int_columns(df_cleaned_copy)
new_df.head(5)

these following columns will be encoded:
- Protocol : vector encoding 
- Packet type : vector encoding 
- Traffic Type : vector encoding
- Malware Indicator : vector encoding
- Alerts/Warnings : vector encoding 
- Attack Type : vector encoding 
- Attack Signature : vector encoding 
- Action Taken : vector encoding 
- Severity Level : ordinal 
- Network Segment : vector encoding
- Firewall Logs : vector encoding
- IDS/IPS Alerts : vector encoding 
- Log Source : vector encoding



In [22]:
column_dtypes_for_new = {   
    "Protocol": "category",
    "Packet Length": int,
    "Packet Type": "category",
    "Traffic Type": "category",
    "Malware Indicators": "category",
    "Anomaly Scores": float,
    "Alerts/Warnings": "category",
    "Attack Type": "category",
    "Attack Signature": "category",
    "Action Taken": "category",
    "Severity Level": "category",
    "Network Segment": "category",
    "Firewall Logs": "category",
    "IDS/IPS Alerts": "category",
    "Log Source": "category",
    #"attack_Index": int
    }

# Convert all other columns based on dictionary
for col, dtype in column_dtypes_for_new.items():
    if dtype == str:
        new_df[col] = new_df[col].astype("string")  # Explicitly enforce str type
    else:
        new_df[col] = new_df[col].astype(dtype)

In [24]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40000 entries, 0 to 39999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Protocol            40000 non-null  category
 1   Packet Length       40000 non-null  int64   
 2   Packet Type         40000 non-null  category
 3   Traffic Type        40000 non-null  category
 4   Malware Indicators  40000 non-null  category
 5   Anomaly Scores      40000 non-null  float64 
 6   Alerts/Warnings     40000 non-null  category
 7   Attack Type         40000 non-null  category
 8   Attack Signature    40000 non-null  category
 9   Action Taken        40000 non-null  category
 10  Severity Level      40000 non-null  category
 11  Network Segment     40000 non-null  category
 12  Firewall Logs       40000 non-null  category
 13  IDS/IPS Alerts      40000 non-null  category
 14  Log Source          40000 non-null  category
dtypes: category(13), float64(1), int64(1)
mem

## Vector encoding 

In [25]:
# Encoding categorical columns using get_dummies
categorical_columns = ['Protocol', 'Packet Type', 'Traffic Type', 'Malware Indicators', 'Alerts/Warnings', 'Attack Signature', 'Action Taken', 'Network Segment', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source']

#df_encoded = new_df.copy()

for col in categorical_columns:
    dummies = pd.get_dummies(new_df[col], prefix=col, dtype=int)
    new_df = pd.concat([new_df, dummies], axis=1)
    #df_encoded = df_encoded.drop(columns=[col])



In [26]:
new_df.head(5)


,Protocol,Packet Length,Packet Type,Traffic Type,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Type,Attack Signature,Action Taken,...,Action Taken_Logged,Network Segment_Segment A,Network Segment_Segment B,Network Segment_Segment C,Firewall Logs_Log Data,Firewall Logs_No Log Data,IDS/IPS Alerts_Alert Data,IDS/IPS Alerts_No Alert Data,Log Source_Firewall,Log Source_Server
attack_Index,,,,,,,,,,,,,,,,,,,,,
0,ICMP,503,Data,HTTP,IoC Detected,28.67,No Alert Triggered,Malware,Known Pattern B,Logged,...,1,1,0,0,1,0,0,1,0,1
1,ICMP,1174,Data,HTTP,IoC Detected,51.50,No Alert Triggered,Malware,Known Pattern A,Blocked,...,0,0,1,0,1,0,0,1,1,0
2,UDP,306,Control,HTTP,IoC Detected,87.42,Alert Triggered,DDoS,Known Pattern B,Ignored,...,0,0,0,1,1,0,1,0,1,0
3,UDP,385,Data,HTTP,No IoC Detected,15.79,Alert Triggered,Malware,Known Pattern B,Blocked,...,0,0,1,0,0,1,1,0,1,0
4,TCP,1462,Data,DNS,No IoC Detected,0.52,Alert Triggered,DDoS,Known Pattern B,Blocked,...,0,0,0,1,0,1,1,0,1,0


## ordinal encoding

In [28]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the "Severity Level" column
new_df['Severity Level'] = label_encoder.fit_transform(new_df['Severity Level'])



In [29]:
# Display the first few rows to verify the encoding
new_df.head(20)

,Protocol,Packet Length,Packet Type,Traffic Type,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Type,Attack Signature,Action Taken,...,Action Taken_Logged,Network Segment_Segment A,Network Segment_Segment B,Network Segment_Segment C,Firewall Logs_Log Data,Firewall Logs_No Log Data,IDS/IPS Alerts_Alert Data,IDS/IPS Alerts_No Alert Data,Log Source_Firewall,Log Source_Server
attack_Index,,,,,,,,,,,,,,,,,,,,,
0,ICMP,503,Data,HTTP,IoC Detected,28.67,No Alert Triggered,Malware,Known Pattern B,Logged,...,1,1,0,0,1,0,0,1,0,1
1,ICMP,1174,Data,HTTP,IoC Detected,51.50,No Alert Triggered,Malware,Known Pattern A,Blocked,...,0,0,1,0,1,0,0,1,1,0
2,UDP,306,Control,HTTP,IoC Detected,87.42,Alert Triggered,DDoS,Known Pattern B,Ignored,...,0,0,0,1,1,0,1,0,1,0
3,UDP,385,Data,HTTP,No IoC Detected,15.79,Alert Triggered,Malware,Known Pattern B,Blocked,...,0,0,1,0,0,1,1,0,1,0
4,TCP,1462,Data,DNS,No IoC Detected,0.52,Alert Triggered,DDoS,Known Pattern B,Blocked,...,0,0,0,1,0,1,1,0,1,0
5,UDP,1423,Data,HTTP,No IoC Detected,5.76,No Alert Triggered,Malware,Known Pattern A,Logged,...,1,0,0,1,0,1,0,1,0,1
6,TCP,379,Data,DNS,No IoC Detected,31.55,No Alert Triggered,DDoS,Known Pattern B,Ignored,...,0,1,0,0,1,0,0,1,0,1
7,ICMP,1022,Data,DNS,IoC Detected,54.05,Alert Triggered,Intrusion,Known Pattern A,Logged,...,1,1,0,0,1,0,1,0,1,0
8,TCP,1281,Control,FTP,IoC Detected,56.34,Alert Triggered,Intrusion,Known Pattern A,Blocked,...,0,0,1,0,1,0,1,0,0,1


## dropping useless columns


In [30]:
columns_to_drop = ['Protocol', 'Packet Type', 'Traffic Type', 'Malware Indicators', 'Alerts/Warnings', 'Attack Signature', 'Action Taken', 'Network Segment', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source']
new_df = new_df.drop(columns=columns_to_drop)
new_df.head(5)

,Packet Length,Anomaly Scores,Attack Type,Severity Level,Protocol_ICMP,Protocol_TCP,Protocol_UDP,Packet Type_Control,Packet Type_Data,Traffic Type_DNS,...,Action Taken_Logged,Network Segment_Segment A,Network Segment_Segment B,Network Segment_Segment C,Firewall Logs_Log Data,Firewall Logs_No Log Data,IDS/IPS Alerts_Alert Data,IDS/IPS Alerts_No Alert Data,Log Source_Firewall,Log Source_Server
attack_Index,,,,,,,,,,,,,,,,,,,,,
0,503,28.67,Malware,1,1,0,0,0,1,0,...,1,1,0,0,1,0,0,1,0,1
1,1174,51.50,Malware,1,1,0,0,0,1,0,...,0,0,1,0,1,0,0,1,1,0
2,306,87.42,DDoS,1,0,0,1,1,0,0,...,0,0,0,1,1,0,1,0,1,0
3,385,15.79,Malware,2,0,0,1,0,1,0,...,0,0,1,0,0,1,1,0,1,0
4,1462,0.52,DDoS,1,0,1,0,0,1,1,...,0,0,0,1,0,1,1,0,1,0


In [16]:
new_df = new_df.drop(columns=['attack_Index']) ## see the part of new df loading making this column the index column 
new_df.head(5)

In [31]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40000 entries, 0 to 39999
Data columns (total 30 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   Packet Length                       40000 non-null  int64   
 1   Anomaly Scores                      40000 non-null  float64 
 2   Attack Type                         40000 non-null  category
 3   Severity Level                      40000 non-null  int64   
 4   Protocol_ICMP                       40000 non-null  int64   
 5   Protocol_TCP                        40000 non-null  int64   
 6   Protocol_UDP                        40000 non-null  int64   
 7   Packet Type_Control                 40000 non-null  int64   
 8   Packet Type_Data                    40000 non-null  int64   
 9   Traffic Type_DNS                    40000 non-null  int64   
 10  Traffic Type_FTP                    40000 non-null  int64   
 11  Traffic Type_HTTP                

# Model Training 


## dataset splitting 

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
df_train, df_test = train_test_split(new_df,test_size = 0.2, random_state=42)

lets see the length of our different datasets 

In [34]:
len(df_train), len(df_test)

(32000, 8000)

In [35]:
print(new_df['Severity Level'].mean())
print(df_train['Severity Level'].mean())
print(df_test['Severity Level'].mean())

1.001325
0.9975625
1.016375


In [36]:
df_train.columns


Index(['Packet Length', 'Anomaly Scores', 'Attack Type', 'Severity Level',
       'Protocol_ICMP', 'Protocol_TCP', 'Protocol_UDP', 'Packet Type_Control',
       'Packet Type_Data', 'Traffic Type_DNS', 'Traffic Type_FTP',
       'Traffic Type_HTTP', 'Malware Indicators_IoC Detected',
       'Malware Indicators_No IoC Detected', 'Alerts/Warnings_Alert Triggered',
       'Alerts/Warnings_No Alert Triggered',
       'Attack Signature_Known Pattern A', 'Attack Signature_Known Pattern B',
       'Action Taken_Blocked', 'Action Taken_Ignored', 'Action Taken_Logged',
       'Network Segment_Segment A', 'Network Segment_Segment B',
       'Network Segment_Segment C', 'Firewall Logs_Log Data',
       'Firewall Logs_No Log Data', 'IDS/IPS Alerts_Alert Data',
       'IDS/IPS Alerts_No Alert Data', 'Log Source_Firewall',
       'Log Source_Server'],
      dtype='object')

In [37]:
df_test.columns

Index(['Packet Length', 'Anomaly Scores', 'Attack Type', 'Severity Level',
       'Protocol_ICMP', 'Protocol_TCP', 'Protocol_UDP', 'Packet Type_Control',
       'Packet Type_Data', 'Traffic Type_DNS', 'Traffic Type_FTP',
       'Traffic Type_HTTP', 'Malware Indicators_IoC Detected',
       'Malware Indicators_No IoC Detected', 'Alerts/Warnings_Alert Triggered',
       'Alerts/Warnings_No Alert Triggered',
       'Attack Signature_Known Pattern A', 'Attack Signature_Known Pattern B',
       'Action Taken_Blocked', 'Action Taken_Ignored', 'Action Taken_Logged',
       'Network Segment_Segment A', 'Network Segment_Segment B',
       'Network Segment_Segment C', 'Firewall Logs_Log Data',
       'Firewall Logs_No Log Data', 'IDS/IPS Alerts_Alert Data',
       'IDS/IPS Alerts_No Alert Data', 'Log Source_Firewall',
       'Log Source_Server'],
      dtype='object')

dividing our target and feature columns

In [38]:
# list of columns features to be used for training
features = [
    'Packet Length', 
    'Anomaly Scores', 
    'Severity Level', 
    'Protocol_ICMP',
    'Protocol_TCP', 
    'Protocol_UDP', 
    'Packet Type_Control',
    'Packet Type_Data', 
    'Traffic Type_DNS', 
    'Traffic Type_FTP',
    'Traffic Type_HTTP', 
    'Malware Indicators_IoC Detected',
    'Malware Indicators_No IoC Detected', 
    'Alerts/Warnings_Alert Triggered',
    'Alerts/Warnings_No Alert Triggered', 
    'Attack Signature_Known Pattern A', 
    'Attack Signature_Known Pattern B',
    'Action Taken_Blocked', 
    'Action Taken_Ignored',
    'Action Taken_Logged',
    'Network Segment_Segment A', 
    'Network Segment_Segment B',
    'Network Segment_Segment C',
    'Firewall Logs_Log Data',
    'Firewall Logs_No Log Data',
    'IDS/IPS Alerts_Alert Data',
    'IDS/IPS Alerts_No Alert Data',
    'Log Source_Firewall',
    'Log Source_Server',
    'Protocol_ICMP',
    'Protocol_TCP',
    'Protocol_UDP',
    'Packet Type_Control',
    'Packet Type_Data',
    'Traffic Type_DNS',
    'Traffic Type_FTP',
    'Traffic Type_HTTP',
    'Malware Indicators_IoC Detected',
    'Malware Indicators_No IoC Detected',
    'Alerts/Warnings_Alert Triggered',
    'Alerts/Warnings_No Alert Triggered',
    'Attack Signature_Known Pattern A', 
    'Attack Signature_Known Pattern B',
    'Action Taken_Blocked', 
    'Action Taken_Ignored', 'Action Taken_Logged',
    'Network Segment_Segment A', 
    'Network Segment_Segment B',
    'Network Segment_Segment C', 
    'Firewall Logs_Log Data',
    'Firewall Logs_No Log Data', 
    'IDS/IPS Alerts_Alert Data',
    'IDS/IPS Alerts_No Alert Data',
    'Log Source_Firewall',
    'Log Source_Server'
]

In [39]:
target_column = 'attack type'

In [55]:
df_train.head(20)

,Packet Length,Anomaly Scores,Attack Type,Severity Level,Protocol_ICMP,Protocol_TCP,Protocol_UDP,Packet Type_Control,Packet Type_Data,Traffic Type_DNS,...,Action Taken_Logged,Network Segment_Segment A,Network Segment_Segment B,Network Segment_Segment C,Firewall Logs_Log Data,Firewall Logs_No Log Data,IDS/IPS Alerts_Alert Data,IDS/IPS Alerts_No Alert Data,Log Source_Firewall,Log Source_Server
attack_Index,,,,,,,,,,,,,,,,,,,,,
14307,729,7.17,Intrusion,2,1,0,0,1,0,0,...,0,0,0,1,0,1,1,0,0,1
17812,1447,54.00,Malware,0,0,1,0,1,0,0,...,1,1,0,0,0,1,1,0,0,1
11020,1119,82.75,Malware,2,0,1,0,0,1,1,...,0,1,0,0,0,1,0,1,1,0
15158,706,86.47,Malware,0,1,0,0,0,1,0,...,1,0,0,1,1,0,1,0,0,1
24990,1202,63.21,Intrusion,0,1,0,0,0,1,0,...,0,0,1,0,1,0,0,1,1,0
5980,1017,12.49,Intrusion,2,0,0,1,0,1,1,...,0,0,0,1,0,1,1,0,0,1
30334,97,41.31,DDoS,1,0,1,0,1,0,0,...,1,1,0,0,0,1,0,1,1,0
26175,485,70.27,DDoS,2,1,0,0,0,1,1,...,1,1,0,0,0,1,0,1,0,1
11195,334,94.28,Malware,0,0,1,0,1,0,1,...,0,1,0,0,0,1,0,1,1,0


###  get the values of the features columns for the training data


In [46]:
X_train = df_train.loc[:,features].values
y_train = df_train['Attack Type'].values

### get the values of the columns for the test data


In [47]:
x_test = df_test.loc[:,features].values
y_test = df_test['Attack Type'].values

## Testing models 

In [48]:
from sklearn.linear_model import LogisticRegression

In [49]:
# here we initialize the model
lr_model = LogisticRegression(random_state=42,max_iter=1500)

In [50]:
# here we train the model on the training data
lr_model.fit(X=X_train, y=y_train)

LogisticRegression(max_iter=1500, random_state=42)

In [52]:
y_test_predicted = lr_model.predict(x_test)
y_test_predicted

array(['Malware', 'Intrusion', 'DDoS', ..., 'DDoS', 'DDoS', 'Intrusion'],
      shape=(8000,), dtype=object)

In [54]:
from sklearn.metrics import confusion_matrix

In [56]:
cf = pd.DataFrame(
    columns=["y_test_DDoS","y_test_Intrusion","y_test_Malware"],index=["y_predicted_DDoS","y_predicted_Intrusion","y_predicted_Malware"]
)

In [58]:
cf.loc[:,:] = confusion_matrix(y_true= y_test,y_pred= y_test_predicted)
cf

,y_test_DDoS,y_test_Intrusion,y_test_Malware
y_predicted_DDoS,1126,625,885
y_predicted_Intrusion,1120,657,944
y_predicted_Malware,1033,643,967


In [59]:
cf/len(y_test)

,y_test_DDoS,y_test_Intrusion,y_test_Malware
y_predicted_DDoS,0.14075,0.078125,0.110625
y_predicted_Intrusion,0.14,0.082125,0.118
y_predicted_Malware,0.129125,0.080375,0.120875


In [60]:
from sklearn.metrics import recall_score, precision_score, f1_score

In [61]:
from sklearn.metrics import classification_report

In [62]:
report =classification_report(y_true=y_test, y_pred=y_test_predicted)

In [63]:
print(report)

              precision    recall  f1-score   support

        DDoS       0.34      0.43      0.38      2636
   Intrusion       0.34      0.24      0.28      2721
     Malware       0.35      0.37      0.36      2643

    accuracy                           0.34      8000
   macro avg       0.34      0.34      0.34      8000
weighted avg       0.34      0.34      0.34      8000



In [66]:
from sklearn.tree import DecisionTreeClassifier

In [67]:
dt_model = DecisionTreeClassifier()

In [68]:
dt_model.fit(X=X_train,y=y_train)

DecisionTreeClassifier()

In [69]:
y_test_predicted_dt = dt_model.predict(x_test)

In [72]:
report_dt = classification_report(y_pred=y_test_predicted_dt,y_true=y_test)
print(report_dt)

              precision    recall  f1-score   support

        DDoS       0.32      0.33      0.33      2636
   Intrusion       0.35      0.34      0.34      2721
     Malware       0.33      0.33      0.33      2643

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000

